In [ ]:
import pandas as pd
import urllib.request
import tarfile
import os
import random
import time
import threading
import datetime
import logging
import copy
from IPython.display import clear_output
import matplotlib.pyplot as plt
from PIL import Image
import signal

In [ ]:
data = "train.csv"
images = pd.read_csv(data)

# enter the target directory name
directory = ""

upper = 50
lower = 30
max_categories = 1066
random_state = 1234
resize_x = 224
resize_y = 224

In [ ]:
def resize(old_path, file, new_path):
    if os.path.isfile(os.path.join(old_path, file)):   
        try:
            img = Image.open(os.path.join(old_path, file))
            img.verify()
            img = Image.open(os.path.join(old_path, file))
            new_img = img.resize((resize_x, resize_y))
        except:
            print("Failed: ", os.path.join(old_path, file))
            return False
        
        if not os.path.exists(new_path):
            os.makedirs(new_path)
        
        new_img.save(os.path.join(new_path, file))
        return True
    else:
        print("error opening file: {}\{}".format(old_path, file))
        return False

In [ ]:
def download(url, path, filename): 
    if not os.path.exists(path):
        os.makedirs(path)
    destination = "{}\{}".format(path, filename)
    
    if os.path.isfile(destination):
        print("{} already exists".format(train_location))
        return False
    
    try: 
        urllib.request.urlretrieve(url, destination)
        resize(path, filename, path)
        print("success: ")
        return True
    except: 
        print("failed: ")
        return False
    finally: 
        print("download {} to {}".format(url, destination))

In [ ]:
def manage_download(selected, backup):
    backup_index = 0
    for index, row in selected.iterrows():
        url = row['url']
        path = "{}\{}".format(directory, row['landmark_id'])
        filename = "{}.jpg".format(row['id'])
        
        # if download failed, loop through the backup table.
        while not download(url, path, filename) and backup_index < len(backup):
            url = backup["url"].iloc[backup_index]
            path = "{}\{}".format(directory, backup["landmark_id"].iloc[backup_index])
            filename = "{}.jpg".format(backup["id"].iloc[backup_index])
            backup_index += 1 

In [ ]:
# get the number of images under each class.
nunique = images.groupby("landmark_id")["id"].nunique()

# select the classes with number of images i, where i >= upper.
full_categories = nunique[nunique >= upper].sample(max_categories, random_state=random_state)

# randomely choose a number of n for each class, where lower < n < upper.
full_categories = full_categories.apply(lambda x: random.randint(lower, upper))

In [ ]:
# for each class
for c, value in full_categories.iteritems():
    print("category: {}".format(c))
    target_size = value
    
    # select the target group of images 
    matched_images = images[images["landmark_id"] == int(c)]
    
    # random sampling
    selected = matched_images.sample(target_size, random_state=random_state)
    
    # select the rest images as the back-up in case of failure of downloading. 
    backup = matched_images[~matched_images['id'].isin(selected['id'])]
    manage_download(selected, backup)
    
    # clear the output.
    clear_output(wait=True)